# Clustering and PCA

### Mushroom Dataset

Podeis obtener el conjunto de datos en el siguiente enlace:

[Mushroom Dataset](https://www.kaggle.com/uciml/mushroom-classification)

Como podréis comprobar, hay muchas variables, todas ellas categóricas, por lo que exploraciones con scatterplot no nos serán útiles como en otros casos.

La variable a predecir ``class`` es binaria.


In [ ]:
# Carga de librerías, las que hemos considerado básicas, añadid lo que queráis :)

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

### Leer conjunto de datos y primer vistazo

In [ ]:
import pandas as pd

# URL del dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"

# Nombres de las columnas según la documentación oficial
column_names = [
    "class", "cap-shape", "cap-surface", "cap-color", "bruises", "odor",
    "gill-attachment", "gill-spacing", "gill-size", "gill-color", "stalk-shape",
    "stalk-root", "stalk-surface-above-ring", "stalk-surface-below-ring",
    "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", "veil-color",
    "ring-number", "ring-type", "spore-print-color", "population", "habitat"
]

# Leer el dataset desde la URL, sin cabeceras, usando los nombres definidos
df = pd.read_csv(url, header=None, names=column_names)

# Mostrar las primeras 5 filas del dataframe
df.head()


### Carga del dataset

Comenzamos leyendo el dataset de setas directamente desde la fuente oficial de la UCI.  
Este archivo no incluye nombres de columna, por lo que se los añadimos manualmente según la documentación del dataset.

Cada fila representa un hongo, y cada columna describe una característica (forma, color, olor, etc.).

La columna `class` indica si el hongo es comestible (`e`) o venenoso (`p`).


### Exploración de datos

In [ ]:
# Mostrar información general del DataFrame
df.info()

# Mostrar número de filas, columnas y valores únicos por variable
print(f"\nNúmero de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")
print("\nValores únicos por variable:")
print(df.nunique())


### Descripción del conjunto de datos

En este bloque obtenemos una visión general del dataset:

- `df.info()` nos muestra cuántos valores hay por columna y el tipo de datos.
- `df.shape` nos indica el número de instancias (filas) y variables (columnas).
- `df.nunique()` nos dice cuántos valores diferentes hay en cada variable, lo cual nos ayuda a entender la variedad en los datos.

Dado que todas las variables son categóricas, es normal que el tipo de dato sea `object`.
Este análisis nos ayuda a detectar posibles columnas poco informativas (con un solo valor) o con demasiados valores únicos.


#### Calcular el número de nulos de cada feature

In [ ]:
# Contar valores nulos estándar (NaN)
print("Valores nulos por columna:\n")
print(df.isnull().sum())

# Contar valores que en realidad son '?' (suelen aparecer como texto en datasets antiguos)
print("\nValores '?' por columna (posibles nulos codificados):\n")
print((df == '?').sum())


### Búsqueda de valores nulos

Primero buscamos valores nulos reales (`NaN`) con `df.isnull().sum()`.

Sin embargo, en muchos datasets antiguos como este, los valores faltantes están representados como símbolos como `'?'`.

Por eso también contamos cuántos `'?'` hay por columna. Esto nos permitirá decidir si imputar, eliminar o transformar esos datos en el preprocesamiento.


#### Buscar valores extraños. Para ello, ver los valores únicos en cada feature

In [ ]:
# Crear un nuevo DataFrame con el número de valores únicos por columna
features_summary = pd.DataFrame({
    "features": df.columns,
    "n_values": df.nunique().values
})

# Mostrar el resumen
features_summary


### Valores únicos por variable

Este resumen nos permite identificar de forma clara cuántos valores diferentes tiene cada variable.

Esto nos ayuda a detectar:

- Variables con un solo valor (poco informativas, probablemente eliminables)
- Variables con muchos valores distintos (pueden requerir codificación especial)


#### Tratar aquellos valores que entendamos que sean nulos


In [ ]:
# Reemplazar los '?' de la columna 'stalk-root' con la moda
modo = df['stalk-root'].mode()[0]
df['stalk-root'] = df['stalk-root'].replace('?', modo)

# Verificar que ya no hay '?'
(df == '?').sum()


### Imputación de valores faltantes

La columna `stalk-root` contiene valores faltantes representados como `'?'`.

Hemos decidido **imputarlos con la moda** (el valor más frecuente de la columna), ya que esto:

- Evita perder datos
- Evita tratar `'?'` como si fuera una categoría real
- Mantiene la consistencia para técnicas como PCA y KMeans


#### Mirad cuántos valores hay en cada feature, ¿Todas las features aportan información? Si alguna no aporta información, eliminadla

In [ ]:
# Identificar columnas con un solo valor único
columnas_constantes = features_summary[features_summary["n_values"] == 1]["features"].tolist()
print("Columnas que se eliminarán por tener un solo valor:")
print(columnas_constantes)

# Eliminar del DataFrame original
df.drop(columns=columnas_constantes, inplace=True)

# Verificar dimensiones
print(f"\nNueva forma del DataFrame: {df.shape}")


### Eliminación de variables poco informativas

Eliminamos las columnas que tienen **un solo valor único**, ya que no aportan variabilidad al modelo.

Estas columnas no ayudan a diferenciar entre clases ni influyen en los componentes principales o en la agrupación de datos.

En nuestro caso, la columna `veil-type` tiene un único valor y por tanto se elimina.


#### Separar entre variables predictoras y variables a predecir

In [ ]:
# Variable objetivo
y = df["class"]

# Variables predictoras
X = df.drop(columns=["class"])

# Comprobamos formas
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")


### Separación de variables predictoras y objetivo

Separamos la variable que queremos predecir (`class`) del resto de variables.

- `y` contiene la columna `class`, que indica si un hongo es comestible (`e`) o venenoso (`p`).
- `X` contiene todas las demás columnas, que usaremos como variables explicativas.

Este paso es fundamental para entrenar modelos supervisados o para evaluar cómo se estructuran los datos sin usar la etiqueta.


#### Codificar correctamente las variables categóricas a numéricas

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Inicializamos el codificador
encoder = OneHotEncoder(sparse_output=False)

# Aplicamos OneHotEncoder a X
X_encoded = encoder.fit_transform(X)

# Convertimos a DataFrame para mayor claridad
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(X.columns))

# Mostramos las primeras filas del nuevo DataFrame codificado
X_encoded_df.head()


### Codificación de variables categóricas

Usamos `OneHotEncoder` para transformar todas las variables categóricas en variables numéricas.

Este proceso convierte cada categoría en una nueva columna binaria (0 o 1), eliminando ambigüedad y permitiendo que algoritmos como PCA y KMeans trabajen correctamente.

Por ejemplo:  
Una columna con tres valores posibles (`rojo`, `verde`, `azul`) se convierte en tres columnas:
- `color_rojo`, `color_verde`, `color_azul`

Este paso aumenta la dimensionalidad, pero es necesario para aplicar técnicas matemáticas a datos categóricos.


#### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

# División del dataset (usamos X e y originales antes de codificar si el modelo requiere categorías)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Comprobamos formas
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")


### División en entrenamiento y test

Dividimos el conjunto de datos en:

- `X_train` / `y_train`: para entrenar modelos
- `X_test` / `y_test`: para evaluar su rendimiento

Reservamos un **33% para test**, usando `random_state=42` para asegurar que todos obtengamos la misma división.

Esto es fundamental cuando comparamos modelos supervisados como Random Forest más adelante.


### ¿Por qué hacemos un Train/Test Split?

En este paso separamos nuestros datos en dos subconjuntos:

- `X_train`, `y_train`: se usan para entrenar los modelos.
- `X_test`, `y_test`: se usan para evaluar cómo se comporta el modelo con datos que **no ha visto** antes.

Esto nos permite medir la capacidad del modelo para generalizar a nuevos datos.

---

### ¿Qué estamos dividiendo?

- `X`: todas las variables predictoras (características del hongo)
- `y`: la clase que indica si el hongo es comestible (`e`) o venenoso (`p`)

---

### ¿Por qué es importante?

Esta división simula un escenario del mundo real:
> Entrenas tu modelo con datos históricos y luego lo usas para hacer predicciones sobre nuevos datos que van llegando.

Además, **evita el sobreajuste (overfitting)**, es decir, que el modelo se aprenda de memoria los datos del entrenamiento y no sepa generalizar.

---

### ¿Por qué `random_state=42`?

El parámetro `random_state` fija la semilla aleatoria para que la partición sea siempre la misma al ejecutar el código.  
Esto permite reproducibilidad: tú y tus compañeros obtendréis el mismo resultado.

---

### Nota

Aunque este paso es fundamental para modelos **supervisados** (como Random Forest),  
también nos permite **comparar resultados con modelos no supervisados**, como clustering, en una fase posterior.


## PCA

Es un conjunto de datos del que aún no hemos visto nada (no tenemos graficas) así que vamos a hacer algunas. Tenemos el problema de que son muchas variables, **PCA al rescate**: le pedimos que nos de dos dimensiones y las pintamos, sabemos que serán **aquellas que retengan más información**.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Codificamos X_train con OneHotEncoder para aplicar PCA
X_train_encoded = encoder.fit_transform(X_train)

# Aplicamos PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_encoded)

# Creamos un DataFrame para graficar más cómodamente
df_pca = pd.DataFrame(data=X_train_pca, columns=["PC1", "PC2"])
df_pca["class"] = y_train.values

# Visualización en scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x="PC1", y="PC2", hue="class", palette={"e": "green", "p": "red"}, data=df_pca, alpha=0.7
)
plt.title("Visualización del conjunto de entrenamiento con PCA (2 componentes)")
plt.xlabel("Primer componente principal (PC1)")
plt.ylabel("Segundo componente principal (PC2)")
plt.legend(title="Clase")
plt.grid(True)
plt.show()


### Reducción de dimensionalidad con PCA

El dataset tiene muchas variables categóricas que, tras ser codificadas con One-Hot Encoding, generan **una matriz de alta dimensión**.

Para poder **visualizar los datos en un plano (2D)**, aplicamos PCA (Análisis de Componentes Principales), una técnica que transforma los datos y reduce su dimensionalidad reteniendo la **mayor varianza posible**.

En este scatter plot representamos los datos proyectados sobre los **dos primeros componentes principales**, coloreando los puntos según la clase (`comestible` o `venenoso`).

Este tipo de visualización nos permite intuir si hay separación o estructura natural entre las clases.


Parece que está bastante separadito, parece que a ojo mucho se puede ver :)

Igualmente, vamos a entrenar un clasificador a ver qué tal lo hace antes de editar más

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Codificamos X_train y X_test con OneHotEncoder
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

# Entrenamos el modelo
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_encoded, y_train)

# Predicciones
y_pred = clf.predict(X_test_encoded)

# Evaluación del modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))


### Clasificador supervisado: Random Forest

Entrenamos un modelo supervisado con Random Forest para tener un **punto de referencia**.

Este modelo intenta aprender la relación entre las características de los hongos (`X_train`) y su clase (`y_train`).

Después evaluamos su rendimiento con los datos de test (`X_test`, `y_test`), midiendo:

- Precisión (`accuracy`)
- Matriz de confusión
- Métricas detalladas por clase (`precision`, `recall`, `f1-score`)

Este resultado nos sirve como referencia para comparar con métodos no supervisados como KMeans.


Es un conjunto sencillo y Random Forest es muy bueno en su trabajo, Igualmente, vamos a ver qué tamaño tenemos de dataset:


In [ ]:
X_train.shape

### Tamaño del conjunto de entrenamiento

El conjunto `X_train` contiene **5443 muestras** y **21 variables predictoras**.

Esto significa que:

- Estamos entrenando nuestros modelos con 5443 hongos distintos, cada uno descrito por 21 características.
- Estas características son variables categóricas como forma, color, superficie del sombrero, olor, etc.
- La columna `class` no se incluye aquí porque ya la hemos separado como variable objetivo (`y_train`).

Es importante conocer esta forma para:

- Evaluar la complejidad del modelo (número de muestras vs número de variables)
- Preparar correctamente el preprocesamiento (por ejemplo, One-Hot Encoding aumentará el número de columnas significativamente)


¿Muchas features no? Vamos a reducir las usando PCA.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Re-codificamos todo el conjunto para trabajar con variables numéricas
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

# Probamos diferentes números de componentes principales
n_features = list(range(2, X_train_encoded.shape[1] + 1, 5))  # Ej: [2, 7, 12, ..., 92]
scores = []

for n in n_features:
    # 1. Reducir dimensionalidad con PCA
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_train_encoded)
    X_test_pca = pca.transform(X_test_encoded)

    # 2. Entrenar clasificador con los datos reducidos
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train_pca, y_train)
    y_pred = clf.predict(X_test_pca)

    # 3. Guardar la precisión
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# Visualizar el rendimiento según el número de componentes
plt.figure(figsize=(10, 6))
sns.lineplot(x=n_features, y=scores, marker="o")
plt.title("Precisión del clasificador vs. número de componentes PCA")
plt.xlabel("Número de componentes")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()


### Reducción de dimensionalidad con PCA y evaluación supervisada

Queremos saber cuántas componentes principales son suficientes para mantener un buen rendimiento del clasificador.

Para ello:

1. Aplicamos PCA sobre los datos con un número creciente de componentes (`n_features`).
2. Entrenamos un modelo de Random Forest sobre esos datos reducidos.
3. Medimos su precisión sobre el conjunto de test.

Este gráfico nos permite identificar un punto de equilibrio:  
el menor número de componentes que mantiene un rendimiento aceptable.

Así reducimos la complejidad del modelo sin sacrificar precisión.


### Análisis de resultados: reducción drástica sin pérdida de precisión

Tras aplicar PCA sobre los datos codificados, observamos que con solo **10 componentes principales** el modelo Random Forest alcanza prácticamente la **misma precisión** que con todas las variables.

Esto significa que:

- Hemos pasado de ~95 columnas (tras OneHotEncoding) a solo 10.
- Estamos utilizando aproximadamente un **10% de las variables transformadas**.
- Incluso hemos reducido más de la mitad respecto a las **21 variables originales** del dataset sin codificar.

Este resultado demuestra que:

✔️ Muchas de las variables codificadas eran **redundantes o poco informativas**.  
✔️ PCA ha sido muy eficaz para condensar la información en menos dimensiones.  
✔️ La reducción de dimensionalidad no solo mejora la eficiencia, sino que **mantiene (o incluso mejora) la capacidad de predicción** del modelo.

Este es un gran ejemplo de cómo el preprocesamiento inteligente mejora los modelos sin necesidad de complejidad extra.


## Clustering

Viendo que el conjunto de datos es sencillito, podemos intentar hacer algo de clustering a ver qué información podemos obtener.

El primer paso va a ser importar la función de Kmeans de sklearn, y a partir de ahi, vamos a buscar el valor óptimo de clusters. Como hemos visto anteriormente, este valor lo obtenemos, por ejemplo, del codo de la gráfica que representa el total de las distancias de los puntos a los centros de los clusters asociados. Os dejo la página de la documentación de sklearn para que lo busquéis:

[K-Means on sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

Con esto solo hay que ahora generar los modelos de kmeans, evaluar y pintar la gráfica para los valores de ``k`` que establezcais.




In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

# Aplicamos PCA para reducir a 2 dimensiones antes de hacer clustering
# (si no lo has hecho ya en una celda anterior)
X_encoded = encoder.fit_transform(X)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_encoded)

# Rango de valores de k (número de clusters a probar)
k_values = range(1, 11)
scores = []

for k in k_values:
    # Definir modelo KMeans
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(X_pca)
    
    # Guardamos la suma de distancias dentro del cluster (inertia)
    scores.append(kmeans.inertia_)

# Representación del método del codo
plt.figure(figsize=(8, 6))
sns.lineplot(x=list(k_values), y=scores, marker="o")
plt.title("Método del codo - Selección del número óptimo de clusters")
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia (suma de distancias a los centroides)")
plt.grid(True)
plt.show()


### Clustering no supervisado con KMeans

Ahora probamos un enfoque **no supervisado**, aplicando `KMeans` sobre los datos reducidos con PCA (2 dimensiones).

El objetivo es descubrir si existen agrupaciones naturales en los datos sin usar la clase (`e` o `p`).

Para seleccionar el número óptimo de clusters (`k`), usamos el **método del codo**:
- Calculamos la **inercia** (suma de distancias de los puntos a sus centros de cluster) para diferentes valores de `k`.
- La gráfica resultante nos permite identificar un “codo” donde el beneficio de añadir más clusters deja de ser significativo.

Este punto suele indicar el número ideal de clusters a usar.


Con el valor que hayáis obtenido de la gráfica, podéis obtener una buena aproximación de Kmeans y con ello podemos pasar a explorar cómo de bien han separado la información los distintos clusters. Para ello, se va a hacer un ``catplot``, seaborn os lo hará solito. Con esto lo que se pretende ver es la distribución de la varaible a predecir en función del cluster que haya determinado Kmeans.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Asegurarse de tener los datos codificados y reducidos
X_encoded = encoder.fit_transform(X)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_encoded)

# Definimos y entrenamos KMeans con k=2
kmeans = KMeans(n_clusters=2, random_state=42, n_init='auto')
clusters = kmeans.fit_predict(X_pca)

# Creamos un nuevo DataFrame con la clase real y el cluster asignado
df_clusters = pd.DataFrame({
    "class": y.values,
    "cluster": clusters
})

# Pintamos con catplot: distribución de clases dentro de cada cluster
ax = sns.catplot(
    col="cluster",
    x="class",
    data=df_clusters,
    kind="count",
    col_wrap=2,
    palette={"e": "green", "p": "red"}
)
ax.fig.suptitle("Distribución de clases reales por cluster (KMeans)", y=1.05)
plt.show()


### Evaluación visual de los clusters

Aunque KMeans no usa la variable `class`, podemos comparar sus resultados con las etiquetas reales para ver si hay alguna relación.

Usamos `sns.catplot` para representar la distribución de clases (`e` o `p`) dentro de cada cluster creado por KMeans.

Esto nos permite evaluar si el algoritmo de clustering ha conseguido **separar razonablemente bien los hongos comestibles y venenosos** sin necesidad de entrenamiento supervisado.

Si un cluster contiene mayoritariamente una sola clase, podemos considerar que la separación es significativa.


Vamos a ver qué tal queda esto pintado. Para ello, repetimos el scatterplot de antes pero usando como color el cluster asignado por kmeans.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Asegúrate de que X_encoded está disponible
X_encoded = encoder.fit_transform(X)

# Reducimos a 2 componentes con PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_encoded)

# Entrenamos KMeans (si no lo has hecho ya)
kmeans = KMeans(n_clusters=2, random_state=42, n_init='auto')
clusters = kmeans.fit_predict(X_pca)

# Creamos un DataFrame para graficar
df_pca_clusters = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca_clusters["cluster"] = clusters

# Visualizamos los clusters en el espacio PCA
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x="PC1", y="PC2", hue="cluster", palette="Set1", data=df_pca_clusters, alpha=0.7
)
plt.title("Clusters obtenidos por KMeans en el espacio PCA")
plt.xlabel("Componente principal 1")
plt.ylabel("Componente principal 2")
plt.legend(title="Cluster")
plt.grid(True)
plt.show()



### Visualización de clusters en espacio PCA

En este scatterplot proyectamos los datos en 2D usando PCA y coloreamos cada punto según el **cluster asignado por KMeans**.

Esto nos permite ver gráficamente **cómo ha agrupado los datos el algoritmo de clustering**, sin necesidad de etiquetas.

Si los clusters aparecen claramente separados, significa que **KMeans ha identificado patrones naturales en los datos**, incluso sin saber qué clase corresponde a cada punto.


¿Es bastante parecido no? No es tan bueno como el Random Forest, pero ha conseguido identificar bastante bien los distintos puntos del dataset sin utilizar las etiquetas. De hecho, el diagrama de factor que hemos visto antes muestra que solo un par de clusters son imprecisos. Si no hubieramos tenido etiquetas esta aproximacion nos hubiera ayudado mucho a clasificar los distintos tipos de hongos.

### ✅ Conclusiones finales

Al comparar los resultados de KMeans (no supervisado) con los de Random Forest (supervisado), observamos lo siguiente:

- El **Random Forest** obtuvo una clasificación excelente, como era de esperar, ya que utiliza las etiquetas durante el entrenamiento.
- El algoritmo **KMeans**, aunque no tiene acceso a las etiquetas (`class`), ha conseguido **agrupar correctamente la mayoría de los puntos** del dataset en dos clusters bien diferenciados.
- El gráfico de `catplot` mostró que **solo algunos clusters tienen mezcla de clases**, pero en general la separación es bastante buena.
- La visualización en el espacio PCA con colores por cluster confirma que los **datos tienen una estructura latente clara**, que puede ser aprovechada incluso sin supervisión.

📌 **Si no hubiéramos tenido etiquetas**, esta aproximación basada en PCA + KMeans nos habría servido para **detectar dos grupos principales** en el dataset de hongos, lo cual ya ofrece un valor muy significativo en tareas de exploración y descubrimiento de patrones.

En resumen:

> Aunque el modelo no supervisado no alcanza la precisión del supervisado, ha demostrado ser una **herramienta útil y potente** para clasificar y entender datos complejos sin necesidad de etiquetas.
